In [ ]:
import numpy as np 
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import os
print(os.listdir("../input"))


In [ ]:
X_train = np.load('../input/beginner/trainbeg.npy')
X_test = np.load('../input/beginner/testbeg.npy')

In [ ]:
Y_train =  np.load('../input/beginner/trainLabels.npy')
Y_train = Y_train.reshape(Y_train.shape[0])
np.squeeze(Y_train)


In [ ]:
inp_dm = X_train.shape[1]
vgg_model_path = '../input/beginner/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5'
from keras.applications import VGG16
conv_base = VGG16(weights=vgg_model_path,include_top=False,input_shape=(inp_dm,inp_dm, 3))
conv_base.trainable = False

for layer in conv_base.layers:
    if layer.name == 'block5_conv1':
        layer.trainable = True
    if layer.name == 'block4_conv1':
        layer.trainable = True    
    else:
        layer.trainable = False

conv_base.summary()

In [ ]:
from keras.models import Sequential, load_model, Model
from keras.layers import Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D, GlobalMaxPooling2D
from keras.layers import Input, BatchNormalization
from keras.layers import Dense

def VGG16_classifier():    
    model = Sequential()
    model.add(conv_base)
    model.add(Flatten()) 
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(30, activation='softmax'))
    model.summary()
    
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

In [ ]:
from keras.callbacks import  ReduceLROnPlateau
from keras.utils import to_categorical
model1 = VGG16_classifier()
callbacks_list = [ReduceLROnPlateau(monitor='loss',factor=0.2,patience=3)]

history_vgg = model1.fit(x = X_train/255.,y = to_categorical(Y_train, num_classes=30),batch_size=64,epochs=100,callbacks = callbacks_list, verbose=1)

In [ ]:
from sklearn.metrics import accuracy_score
y_train_predict = np.argmax(model1.predict(x=X_train/255.),axis = 1)

print('\n',y_train_predict)

np.squeeze(Y_train)

print(Y_train)

print("Train accuracy : {}%".format(accuracy_score(Y_train,y_train_predict)))

In [ ]:


y_test_predict = model1.predict(x=X_test/255.)

print('\n',y_test_predict)



In [ ]:
data_classes = ["antelope","bat","beaver","bobcat","buffalo","chihuahua","chimpanzee","collie","dalmatian","german+shepherd","grizzly+bear",
                "hippopotamus","horse","killer+whale","mole","moose","mouse","otter","ox","persian+cat","raccoon","rat","rhinoceros","seal",
                "siamese+cat","spider+monkey","squirrel","walrus","weasel","wolf"]
label_df = pd.DataFrame(data=y_test_predict, columns= data_classes)

subm = pd.DataFrame()


te_label = pd.read_csv('../input/beginner/test.csv')

subm['image_id'] = te_label['Image_id']

subm = pd.concat([subm, label_df], axis=1)

subm.to_csv('vgg.csv',index = False)

In [ ]:
table = subm.drop(columns=['image_id'])
final_output = table.idxmax(axis=1)
pd.set_option('display.max_rows', 6000)
predictions = []
for i in final_output:
    predictions.append(i)

In [ ]:
import pandas as pd
gold_labels = []
gold = pd.read_csv("../input/goldlabels/gold-2.csv")
for i in gold.Label:
    gold_labels.append(i)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(gold_labels, predictions))

In [ ]:
print("Test accuracy : {}%".format(accuracy_score(gold_labels, predictions)))